In [5]:
import numpy as np
from typing import Tuple, Mapping
import random
from numba import njit
import timeit

In [6]:
@njit
def Booth(x: np.array, y:np.array) -> np.float128:
  return (x + 2 * y - 7) * (x + 2 * y - 7) + (2 * x + y - 5) * (2 * x + y - 5)
@njit
def Booth_dfdx(x: np.array, y:np.array) -> np.float128:
  return 10 * x + 8 * y - 34
@njit
def Booth_dfdy(x: np.array, y:np.array) -> np.float128:
  return 10 * y + 8 * x - 38

In [7]:
@njit
def my_GD(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float, lr: float = 0.01,
          T:int = 1000) -> Tuple [np.ndarray, np.float32]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  lr - скорость обучения (default: 0.01)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0

  for i in range(T):
    xt = xt - lr * dx(xt, yt)
    yt = yt - lr * dy(xt, yt)
  return x0, y0, xt, yt, f(xt, yt)

In [8]:
x0, y0, x, y, z = my_GD(Booth, Booth_dfdx, Booth_dfdy, 10, -10, lr = 0.1)
print("Градиентный спуск\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", x, ", yt =", y,
      "\nЗначение функции в точке глобального минимума: f(xt, yt) =", z)

print(timeit.timeit("my_GD(Booth, Booth_dfdx, Booth_dfdy, 10, -10, lr = 0.1)", globals=globals())) #with njit

Градиентный спуск
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = 1.0000000000000002 , yt = 2.9999999999999996 
Значение функции в точке глобального минимума: f(xt, yt) = 7.888609052210118e-31
51.351510649000005


In [9]:
print(timeit.timeit("my_GD(Booth, Booth_dfdx, Booth_dfdy, 10, -10, lr = 0.1)", globals=globals())) #without njit

50.68821549299999


In [10]:
@njit
def my_GD_Momentum(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float, 
        beta: float = 0.9, T:int = 1000) -> Tuple [int, int, float, float, float]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  beta - коэффициент обучения (default: 0.9)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0

  for i in range(T):
    xt = xt * beta - (1 - beta) * dx(xt, yt)
    yt = yt * beta - (1 - beta) * dy(xt, yt)
  return x0, y0, xt, yt, f(xt, yt)

In [11]:
x0, y0, p_x, p_y, p_z = my_GD_Momentum(Booth, Booth_dfdx, Booth_dfdy, x0=10, y0=-10)
print("Алгоритм GD Momentum\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", p_x, ", yt =", p_y,
      "\nГлобальный минимум: f(xt, yt) =", p_z)

Алгоритм GD Momentum
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = 1.2280701754385972 , yt = 2.561403508771929 
Глобальный минимум: f(xt, yt) = 0.4216682056017245


In [12]:
@njit
def my_GD_ADAM(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float,
        lr: float = 0.1, beta1: float = 0.9, beta2: float = 0.99,
        eps: float = 1e-8, T:int = 1000) -> Tuple [int, int, float, float, float]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  lr - коэффициент скорости обучения (default: 0.1)
  beta1 - параметр (default: 0.9)
  beta2 - параметр (default: 0.99)
  eps - параметр (default: 1e-8)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0
  vtx = 0
  vty = 0
  Gtx = 0
  Gty = 0

  for i in range (T):
    grad_x = dx(xt, yt)
    grad_y = dy(xt, yt)
    vtx = beta1 * vtx + (1 - beta1) * grad_x
    vty = beta1 * vty + (1 - beta1) * grad_y
    Gtx = beta2 * Gtx + (1 - beta2) * (grad_x) ** 2
    Gty = beta2 * Gty + (1 - beta2) * (grad_y) ** 2
    xt = xt - lr * vtx / (Gtx + eps) ** (1 / 2)
    yt = yt - lr * vty / (Gty + eps) ** (1 / 2)
  return x0, y0, xt, yt, f(xt, yt)

In [13]:
x0, y0, p_x, p_y, p_z = my_GD_ADAM(Booth, Booth_dfdx,Booth_dfdy, x0=10, y0=-10)
print("Алгоритм GD ADAM\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", p_x, ", yt =", p_y,
      "\nГлобальный минимум: f(xt, yt) =", p_z)

Алгоритм GD ADAM
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = 1.0 , yt = 3.0000000000000018 
Глобальный минимум: f(xt, yt) = 1.5777218104420236e-29
